Players dataset creation

The Fantacalcio players list is manually downloaded from https://www.fantacalcio.it/quotazioni-fantacalcio

Here, the players database is generated, by merging the Fantacalcio list to stats downloaded from http://fbref.com

In [1]:
import pandas as pd

Load fbref data for outfield players and goalkeepers.

Generate fbref player list, adding player surname (with special characters replaced to normal ones)

In [2]:
rcsv = pd.read_csv('fbref_data/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [3]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)

players

,player,team
0,Tammy Abraham,Roma
1,Francesco Acerbi,Inter
2,Yacine Adli,Milan
3,Michel Aebischer,Bologna
4,Felix Afena-Gyan,Cremonese
...,...,...
515,Wojciech Szczęsny,Juventus
516,Ciprian Tătărușanu,Milan
517,Pietro Terracciano,Fiorentina
518,Guglielmo Vicario,Empoli


In [4]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [5]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [6]:
print(players[['player', 'surname']].to_string())

                         player           surname
0                 Tammy Abraham           Abraham
1              Francesco Acerbi            Acerbi
2                   Yacine Adli              Adli
3              Michel Aebischer         Aebischer
4              Felix Afena-Gyan        Afena-Gyan
5                 Kevin Agudelo           Agudelo
6                      Ola Aina              Aina
7                 Emanuel Aiwum             Aiwum
8        Jean-Daniel Akpa-Akpro        Akpa-Akpro
9                  Luis Alberto           Alberto
10     Agustín Álvarez Martínez          Martinez
11                 Kelvin Amian             Amian
12                 Bruno Amione            Amione
13                 Bruno Amione            Amione
14                 Ethan Ampadu            Ampadu
15               Sofyan Amrabat           Amrabat
16              Felipe Anderson          Anderson
17                Janis Antiste           Antiste
18               Marcos Antônio           Antonio


Replace the surname for some specific players, according to config/name_fix.txt file.

This is done for players for which the decoded fbref surname doesn't correspond to Fantacalcio list.

In [7]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix



Ostigard
Kim
Hojlund
Gytkjaer
Augusto
Maehle
Kjaer
Djuricic
Djuric
Cabral
Alvarez


,FROM,TO,TEAM
0,stigard,Ostigard,Napoli
1,Min-jae,Kim,Napoli
2,Hjlund,Hojlund,Atalanta
3,Gytkjr,Gytkjaer,Monza
4,Carlos,Augusto,Monza
5,Mhle,Maehle,Atalanta
6,Kjr,Kjaer,Milan
7,uricic,Djuricic,Sampdoria
8,uric,Djuric,Hellas Verona
9,Arthur,Cabral,Fiorentina


Load players from Fantacalcio list.

In [8]:
fc_data = pd.read_excel('fantacalcio/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players['surname'][i] = spl[-2]
        fc_players['initial'][i] = spl[-1][0]
    else:
        fc_players['surname'][i] = spl[-1]
        fc_players['initial'][i] = ''
    
fc_players



C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\2263921821.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-1]
C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\2263921821.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['initial'][i] = ''
C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\2263921821.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-2]


,id,r,name,team,surname,initial
0,572,P,Meret,Napoli,Meret,
1,2814,P,Provedel,Lazio,Provedel,
2,4312,P,Maignan,Milan,Maignan,
3,2211,P,Silvestri,Udinese,Silvestri,
4,159,P,Sepe,Salernitana,Sepe,
...,...,...,...,...,...,...
549,5512,A,De Luca,Sampdoria,Luca,
550,5734,A,Soule',Juventus,Soule,
551,5785,A,Lazetic,Milan,Lazetic,
552,5837,A,Voelkerling Persson,Lecce,Persson,


Associate players from Fantacalcio list to ID for FBref data.

In [9]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players['fb_ID'][i] = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                fc_players['fb_ID'][i] = j
                
        

C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\1456862974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = -1
C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\1456862974.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = j


Print players for which the association failed.

Most of them are players who didn't play a single Serie A game this season. 

Others are ones for which the FBRef surname doesn't correspond to Fantacalcio one.

For example, Cabral is Arthur for FBref.

Correction is made in the name_fix code above.

In [10]:
for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        print(fc_players['name'][i])

Berisha
Cragno
Mirante
Sarr M.
Lamanna
Ujkani
Pegolo
Perilli
Padelli
Perisan
Bardi
Cordaz
Pinsoglio
Fiorillo
Sirigu
Cerofolini
Rossi F.
Contini
Brancolini
Bleve
Berardi A.
Russo A.
Gemello
Ravaglia
Boer
Adamonis
Marfella
Zovko
Piana
Bagnolini
Svilar
Sorrentino A.
Ciezkowski
Micai
Chiesa M.
Saro
Palomino
Aiwu
Tonelli
Radu
Paletta
Fares
Romagna
Ferrer
Amey
Ferrarini
Kamenovic
Zanotti
Motoc
Bayeye
Buta
Ndiaye
Abankwah
Guessand A.
Guarino
Pogba
Winks
Castrovilli
Capezzi
Machin
Bakayoko
Scozzarella
Akpa Akpro
Darboe
Urbanski
Bertini
Bianco
Sher
Nguiamba
Praszelik
Trimboli
Pafundi
Bjorkengren
Samek
Acella
Tripi
Garbett
Ibrahimovic
Edera
Oddei
Raimondo
Kaio Jorge
Voelkerling Persson


In [11]:
fc_players

,id,r,name,team,surname,initial,fb_ID
0,572,P,Meret,Napoli,Meret,,502
1,2814,P,Provedel,Lazio,Provedel,,509
2,4312,P,Maignan,Milan,Maignan,,500
3,2211,P,Silvestri,Udinese,Silvestri,,512
4,159,P,Sepe,Salernitana,Sepe,,511
...,...,...,...,...,...,...,...
549,5512,A,De Luca,Sampdoria,Luca,,122
550,5734,A,Soule',Juventus,Soule,,422
551,5785,A,Lazetic,Milan,Lazetic,,248
552,5837,A,Voelkerling Persson,Lecce,Persson,,-1


Populate players dataset with stats from FBref, for outfield players and goalkeepers

In [12]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            

C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

In [13]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [14]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
         for j in range(columns_to_copy.shape[0]):
            fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]

C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_2484\3664433845.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [15]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,572,P,Meret,Napoli,Meret,,502,25-234,1997,0,...,26.9,92,19.6,26.6,142,6,4.2,10,0.71,15.1
1,2814,P,Provedel,Lazio,Provedel,,509,28-239,1994,0,...,33.1,94,37.2,36.6,201,5,2.5,29,2.08,17.6
2,4312,P,Maignan,Milan,Maignan,,500,27-131,1995,0,...,33.5,18,38.9,40.4,65,5,7.7,6,0.86,12.9
3,2211,P,Silvestri,Udinese,Silvestri,,512,31-254,1991,0,...,34.4,110,40.9,38.2,194,4,2.1,5,0.36,11.3
4,159,P,Sepe,Salernitana,Sepe,,511,31-187,1991,0,...,36.6,128,48.4,41.7,187,14,7.5,9,0.64,13.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,5512,A,De Luca,Sampdoria,Luca,,122,24-117,1998,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
550,5734,A,Soule',Juventus,Soule,,422,19-210,2003,5,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
551,5785,A,Lazetic,Milan,Lazetic,,248,18-293,2004,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
552,5837,A,Voelkerling Persson,Lecce,Persson,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


Load votes database, to add data to players database (mean vote and its standard deviation)

In [16]:
import numpy as np

votes = pd.read_excel('mid_outputs/players_votes.xlsx', index_col = 0)

Compute the average Serie A Goal Keeper mean vote and vote std

In [17]:
min_votes = 6

perf_df_P = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]): 
    if(fc_players.loc[i]['r'] == 'P'):
        v = np.array([])
        for j in range(votes.shape[0]):
            if(fc_players['name'][i] == votes['player'][j]):
                v = np.append(v, votes['vote'][j])

        if(v.shape[0] >= min_votes - 1):
            row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df_P.columns)
            perf_df_P = pd.concat([perf_df_P, row_df], ignore_index = True)


print(perf_df_P.mean())

perf_df_P


vote_avg    6.214464
vote_std    0.438749
dtype: float64


,vote_avg,vote_std
0,6.214286,0.557875
1,6.321429,0.305143
2,6.357143,0.440315
3,6.321429,0.485767
4,6.428571,0.371154
5,6.125000,0.216506
6,6.142857,0.349927
7,6.428571,0.371154
8,6.187500,0.347985
9,6.428571,0.416497


Add to players data their mean vote (and its standard deviation).

For players who don't have a minimum amount of games, more data to reach this value is computed, according to the average Serie A player vote (and std). For goalkeepers, this values are different.


In [18]:
min_votes = 6

#outfield players
mean_def = 6
std_def = 0.58

#goalkeepers
mean_def_P = 6.22
std_def_P = 0.43


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    mean_def_i = mean_def
    std_def_i = std_def
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

,vote_avg,vote_std
0,6.214286,0.557875
1,6.321429,0.305143
2,6.357143,0.440315
3,6.321429,0.485767
4,6.428571,0.371154
...,...,...
549,5.877688,0.485814
550,6.067846,0.581002
551,6.170097,0.204814
552,6.041983,0.689071


In [19]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,572,P,Meret,Napoli,Meret,,502,25-234,1997,0,...,19.6,26.6,142,6,4.2,10,0.71,15.1,6.214286,0.557875
1,2814,P,Provedel,Lazio,Provedel,,509,28-239,1994,0,...,37.2,36.6,201,5,2.5,29,2.08,17.6,6.321429,0.305143
2,4312,P,Maignan,Milan,Maignan,,500,27-131,1995,0,...,38.9,40.4,65,5,7.7,6,0.86,12.9,6.357143,0.440315
3,2211,P,Silvestri,Udinese,Silvestri,,512,31-254,1991,0,...,40.9,38.2,194,4,2.1,5,0.36,11.3,6.321429,0.485767
4,159,P,Sepe,Salernitana,Sepe,,511,31-187,1991,0,...,48.4,41.7,187,14,7.5,9,0.64,13.7,6.428571,0.371154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,5512,A,De Luca,Sampdoria,Luca,,122,24-117,1998,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.877688,0.485814
550,5734,A,Soule',Juventus,Soule,,422,19-210,2003,5,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.067846,0.581002
551,5785,A,Lazetic,Milan,Lazetic,,248,18-293,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.170097,0.204814
552,5837,A,Voelkerling Persson,Lecce,Persson,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.041983,0.689071


In [20]:
fc_players.columns[123]

'gk_games'

For goalkeepers who didn't play a miminum amount of games, data is weightly averaged with one of the main goalkeeper of their same team.

In [21]:
min_gk_games = 6

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[123:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.at[i, columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

Gollini, 0.5
Berisha, 0.0
Cragno, 0.0
Luis Maximiano, 0.16666666666666663
Carnesecchi, 0.8333333333333334
Mirante, 0.0
Sarr M., 0.0
Lamanna, 0.0
Ujkani, 0.0
Pegolo, 0.0
Perilli, 0.0
Padelli, 0.0
Perisan, 0.0
Bardi, 0.0
Cordaz, 0.0
Pinsoglio, 0.0
Fiorillo, 0.0
Sirigu, 0.0
Cerofolini, 0.0
Rossi F., 0.0
Contini, 0.0
Brancolini, 0.0
Bleve, 0.0
Berardi A., 0.0
Russo A., 0.0
Gemello, 0.0
Ravaglia, 0.0
Zoet, 0.16666666666666663
Boer, 0.0
Adamonis, 0.0
Marfella, 0.0
Zovko, 0.0
Piana, 0.0
Bagnolini, 0.0
Svilar, 0.0
Sorrentino A., 0.0
Ciezkowski, 0.0
Micai, 0.0
Chiesa M., 0.0
Saro, 0.0


In [22]:
fc_players = fc_players_newgk

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,572,P,Meret,Napoli,Meret,,502,25-234,1997,0,...,19.6,26.6,142.0,6.0,4.2,10.0,0.71,15.1,6.214286,0.557875
1,2814,P,Provedel,Lazio,Provedel,,509,28-239,1994,0,...,37.2,36.6,201.0,5.0,2.5,29.0,2.08,17.6,6.321429,0.305143
2,4312,P,Maignan,Milan,Maignan,,500,27-131,1995,0,...,38.9,40.4,65.0,5.0,7.7,6.0,0.86,12.9,6.357143,0.440315
3,2211,P,Silvestri,Udinese,Silvestri,,512,31-254,1991,0,...,40.9,38.2,194.0,4.0,2.1,5.0,0.36,11.3,6.321429,0.485767
4,159,P,Sepe,Salernitana,Sepe,,511,31-187,1991,0,...,48.4,41.7,187.0,14.0,7.5,9.0,0.64,13.7,6.428571,0.371154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,5512,A,De Luca,Sampdoria,Luca,,122,24-117,1998,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,5.877688,0.485814
550,5734,A,Soule',Juventus,Soule,,422,19-210,2003,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.067846,0.581002
551,5785,A,Lazetic,Milan,Lazetic,,248,18-293,2004,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.170097,0.204814
552,5837,A,Voelkerling Persson,Lecce,Persson,,-1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.041983,0.689071


Save to file.

In [23]:
fc_players.to_excel('mid_outputs/players_stats.xlsx')